In [1]:
# ============================================================
# Notebook setup
# ============================================================

%load_ext autoreload
%autoreload 2

# Control figure size
interactive_figures = False
if interactive_figures:
    # Normal behavior
    %matplotlib widget
    figsize=(9, 3)
else:
    # PDF export behavior
    figsize=(14, 4)

from util import util

# Column Generation for Better Scalability

## Scalability Issues

**The main drawback of the cycle formulation is the limited scalability**

* The number of cycles grows with the graph size as $O(n^{\text{max length}})$
* The enumeration becomes _more expensive_ and the model becomes _larger_

**Both can quickly become major bottlenecks**

In [2]:
pairs2, arcs2, aplus2 = util.generate_compatibility_graph(size=150, seed=2)
print('>>> Size 150, enumeration time')
%time cycles2 = util.find_all_cycles(aplus2, max_length=4, cap=None)
print(f'Number of cycles: {len(cycles2)}')
print('>>> Size 150, solution time')
%time _, _, _ = util.cycle_formulation(pairs2, cycles2, tlim=10, verbose=0)

>>> Size 150, enumeration time
CPU times: user 7.34 s, sys: 10 ms, total: 7.35 s
Wall time: 7.35 s
Number of cycles: 43206
>>> Size 150, solution time
CPU times: user 1.57 s, sys: 63.5 ms, total: 1.64 s
Wall time: 1.65 s


<div class=big>
    <small>Essentially, we have too many variables?</small><br>
    How can we address this?
</div>

## Incremental Addition of Variables

**We'll see how we can _introduce variables as needed_**

Let us assume we have an optimization problem over a non-negative variable:

$$
\mathop{\rm argmin}_{x  \geq 0} f(x)
$$

* We assume that $f(x)$ is _convex_, which would make the problem easy
* ...Except that $x$ is so _large-dimensional_ that we cannot scale

**We could obtain a solution quickly as follows:**

* We restrict to a small subset $S$ of the components of $x$
* ...We fix to 0 all other variables, i.e. $x_j = 0$ if $j \notin S$
* ...Then we find an optimum $x^*_S$ via any suitable approach

<div class=big>
But how do we know whether $x^*_S$ is optimal?
</div>

## Pricing

**We know that $x^*_S$ is optimal w.r.t. variables in $S$**

For all the _other_ $x_j$, we can test a condition _after_ the problem is solved:

$$
\frac{\partial f(x^*_S)}{\partial x_j} \geq 0 \qquad \forall j \notin S
$$

**We know that:**

* The cost function is convex
* All variables $\notin S$ are forced to 0 $\Rightarrow$ if we add them, they can only _increase_

**Therefore:**

* If $\partial f(x^*_S) / \partial x_j \geq 0$, then adding the variable cannot be beneficial
* If $\partial f(x^*_S) / \partial x_j < 0$, then adding the variable may improve the cost

**This post-solution derivative check is sometimes called _pricing_**

## Pricing for Incremental Variable Addition

**So, we _a criterion_ to find which variables should be added**

In principle, we could proceed as follows:

* Choose $S$ and we solve over $x_S$
* Loop over all $j \notin S$ and check $\partial f(x^*) / \partial x_j$
* Add the non-optimal variables to $S$ and repeat until $\nabla_x f(x^*) \geq 0$

**This is a nice method, but it has one potential weakness:**

* If we need to  _enumerate_ to check $\partial f(x^*) / \partial x_j$
* ...That may still take way too much time

> **We need a way to do the derivative check more efficiently**

## From Variable Addition to Variable Generation

**Let's focus on decision variables representing complex entities**

...Which can be constructed based on simpler building blocks

* E.g. cycles including several _nodes_
* E.g. routes including several _arcs_

**We can formalize this situation as follows:** 

$$
x_j \equiv g(y_j)
$$

Given a variable $y_j$ that specifies which building blocks are used:

* E.g. which nodes are included in the $j$-th cycle
* E.g. which arcs are included in the $j$-th route

...The function $g(y)$ specifies how a$x_j$ is built

## Pricing Problem

**In these cases, we can avoid enumeration by using optimization**

* First, we compute _in closed form_ the derivative $\partial f(x^*_S) / \partial g(y)$
* Then we solve the _pricing problem_:

$$
y^* = \mathop{\rm argmin}_{y} \frac{\partial f(x^*_S)}{\partial g(y)}
$$

**The result is the "recipe" $y^*$ for the best possible variable $g(y^*)$**

* I.e. the set of nodes leading to the best cycle
* I.e. the set of arcs leading to the best route

...And we can check the corresponding partial derivative as before

## Column Generation

**Let's revisit the process with this change**

* Choose $S$ and solve a _restricted master problem_ over $x_S$
* Solve the pricing problem:
$$
y^* = \mathop{\rm argmin}_{y} \frac{\partial f(x^*_S)}{\partial g(y)}
$$
* If $\partial f(x^*_S) / g(y^*) \geq 0$: the solution is optimal
* Oherwise: add $g(y^*)$ to the set variables and repeat

**This process is called variable generation, or more often _column generation_**

...Because in Linear Programming variables are associated to columns

<div class=big>
    <small>The cycle formulation has a lot of variables...</small><br>
    Can we use this method for our use case?
</div>

## CG for the KEP

**We can try to use it for the _relaxed_ cycle formulation:**

$$\begin{align}
\min & - \sum_{j=1}^n w_j x_j \\
\text{s.t. } & \sum_{j=1}^n a_{ij} x_j \leq 1 & \forall i = 1..m \\
& x_j \geq 0 & \forall j = 1..n
\end{align}$$

* All _integrality_ constraints are _relaxed_, making the problem _convex_
* Note that $x_j \leq 1$ is implied by the other problem constraints
* We also changed the optimization direction to match the CG theory

## CG for the KEP

**Now we need to specify several things**

* How to solve the restricted _master problem_
* How to compute the _partial derivative_ $\partial f(x^*_S) / \partial x_j$
* Which _basic decisions $y$_ to use for constructing a solution
* How those decision affect _the variable buing built_, i.e. the $g(y)$ function
* Finally, we need to define the _pricing problem_

> **We'll tackle one step at a time**

## CG for the KEP: Restricted Master Problem

**Solving this problem for a subset $S$ of variable is easy:**

$$\begin{align}
\min & - \sum_{\color{red}{j \in S}} w_j x_j \\
\text{s.t. } & \sum_{\color{red}{j \in S}} a_{ij} x_j \leq 1 & \forall i = 1..m \\
& x_j \geq 0 & \forall j = 1..n
\end{align}$$

* Rather than setting all other variables to 0
* We just restrict the summations

This is _much cheaper_ in terms of memory usage and solution time

## CG for the KEP: Partial Derivative

**The tricky part is computing $\partial f(x^*_S) / \partial x_j$**

At a first glance, this seems easy:

* By differentiating:

$$
- \sum_{j = 1}^n w_j x_j
$$

* We simply get:

$$
- w_j
$$

* However, unlike in our theoretical formulation
* ...Our problem has _additional constraints_

## CG for the KEP: Partial Derivative

**In particular, we have node mutual exclusion**

$$\begin{align}
\min & - \sum_{j = 1}^n w_j x_j \\
\text{s.t. } & \color{red}{\sum_{j = 1}^n a_{ij} x_j \leq 1} & \forall i = 1..m \\
& x_j \geq 0 & \forall j = 1..n
\end{align}$$

* In the optimal solution, some gradient component can be $> 0$
* ...Because the constraint prevents from moving in that diretion

> **How can we account for this?**

## CG for the KEP: Partial Derivative

**Linear Programs satisfy _strong duality_**

This means that the constraints can be _turned into cost terms_:

$$\begin{align}
\min \mathcal{L}(x, \lambda) = & - \sum_{j = 1}^n w_j x_j + \sum_{i=1}^m \lambda_i \left( \sum_{j \in S}^n a_{ij} x_j - 1 \right) \\
\text{s.t. } & x_j \geq 0  \qquad \forall j = 1..n
\end{align}$$

The new cost function $\mathcal{L}(x, \lambda)$ is called a _Lagrangian_

* It is possible to define _Lagrangian (or dual) multipliers $\lambda_i$_
* S.t. $\nabla \mathcal{L}$ behaves like a normal gradient for an optimal solution

**In fact, all LP solvers are capable to returning those $\lambda$**

## CG for the KEP: Partial Derivative

**So, we differentiate $\mathcal{L}$ rather than the original cost**

$$
\frac{\partial \mathcal{L}(x^*_S)}{\partial x_j} = -w_j + \sum_{i=1}^m \lambda_i^* a_{ij}
$$

* Where $\lambda_i^*$ are the optimal multipliers for the $x^*_S$ solution
* Again, they are computed automatically by the solver

I.e. this derivative is for one specific solution!

**This partial derivative is called a _reduced cost_**

* Reduced costs can be computed by using standard formulas
* ...But here we have derived them step by step

## CG for the KEP: Building Variables

**Now we need to specify how to build a variable, i. the $g(y)$ function**

$$\begin{align}
\min & - \sum_{j = 1}^n \color{red}{w_j} x_j \\
\text{s.t. } & \sum_{j = 1}^n \color{red}{a_{ij}} x_j \leq 1 & \forall i = 1..m \\
& x_j \geq 0 & \forall j = 1..n
\end{align}$$

The basic decision $y_i$ consists in choosing whether to include node $i$

* When we set $y_i = 1$, for a previously unused variable $x_j$
* ...We increase $w_j$ by 1 and we set $a_{ij} = 1$

## CG for the KEP: Pricing Problem

**The goal of the pricing problem is to minimize $\partial f(x^*_S) / \partial g(y)$**

In practice this mean computeing the reduced cost:

$$
\frac{\partial f(x^*_S)}{\partial x_j} = -w_j + \sum_{i=1}^m \lambda_i^* a_{ij}
$$

...Expressed _as a function of $y$_:

$$
\frac{\partial f(x^*_S)}{\partial g(y)} = - \underbrace{\sum_{i=1}^m y_i}_{w_j} + \sum_{i=1}^m \lambda_i^* y_i
$$

* This is true since we can _decide_ which nodes to include

## CG for the KEP: Pricing Problem

**Overall, our pricing problem is as follows:**

$$\begin{align}
\mathop{\rm argmin} & \sum_{i=1}^m y_i (- 1 + \lambda_i^*) \\
\text{s.t. } & y \text{ defines a cycle} \\
& \sum_{i=1}^m y_i \leq C \\
& y_i \in \{0, 1\} & \forall i = 1..m
\end{align}$$

* Our selection nodes should have minimal weight
* ...It should define a cycle
* ...And it should not be too large

## Column Generation

**Overall, this is how CG is set up for Linear Programs**

...Which are by far the most common application case:


<center style="padding-top: 1em; padding-bottom: 1em"><img src="assets/cg.png" width="70%"/></center>

* After every MP iteration, we obtain the dual multiplier
* ...Then we solve the pricing problem to obtain the best possible variable
* If the corresponging reduced cost is $\geq 0$, we proved optimality
* Otherwise, we keep on looping